In [ ]:
# In[1]:
import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
# %%
# define function
import src.SAT_function as data_process
import src.Data_Preprocess as preprocess

In [ ]:
import src.slurm_cluster as scluster
client, scluster = scluster.init_dask_slurm_cluster()

In [ ]:
def func_mk(x):
    """
    Mann-Kendall test for trend
    """
    results = data_process.mk_test(x)
    slope = results[0]
    p_val = results[1]
    return slope, p_val
 

In [ ]:
# Input the MMEM annual mean SAT data
input_model = './Datafiles/'
IPSL_data     = xr.open_mfdataset(input_model + 'tas_IPSL_annual_ano_1850_2022.nc',chunks = {'run':1})

In [ ]:
IPSL_data

### Apply the MMEs GSAT timeseries to extract forced and unforced component of GSAT

In [ ]:
# input GSAT time series
input_ts_dir = './Figure1/'

GSAT_IPSL_annual = xr.open_dataset(input_ts_dir + 'GMSAT_IPSL_CM6A_annual_timeseries_ENS.nc')
GSAT_IPSL_annual

In [ ]:
IPSL_data['tas'].values.shape

In [ ]:
slope_Forced_signal, intercept_Forced_signal = data_process.linear_regression_single_gmst_multi_sat(
    GSAT_IPSL_annual['tas'],
    IPSL_data['tas'].values)

In [ ]:
# separate the GSAT into forced and internal variability
lat = IPSL_data['lat']
lon = IPSL_data['lon']
run = IPSL_data['run']

In [ ]:
slope_Forced_signal.shape

In [ ]:
slope_Forced_signal_da = xr.DataArray(slope_Forced_signal, coords=[run,lat, lon], dims=['run','lat', 'lon'])
intercept_Forced_signal_da = xr.DataArray(intercept_Forced_signal, coords=[run, lat, lon], dims=['run','lat', 'lon'])

In [ ]:
intercept_Forced_signal_da

In [ ]:
# reconstruct the forced signal
GSAT_IPSL_Forced_signal = GSAT_IPSL_annual['tas'] * slope_Forced_signal_da  + intercept_Forced_signal_da

In [ ]:
# reorder the dimensions
GSAT_IPSL_Forced_signal = GSAT_IPSL_Forced_signal.transpose('run','year','lat','lon') 

In [ ]:
GSAT_IPSL_Forced_signal

In [ ]:
# calculate the internal variability
GSAT_IPSL_Internal_variability = IPSL_data - GSAT_IPSL_Forced_signal

In [ ]:
GSAT_IPSL_Internal_variability

In [ ]:
# output the forced and unforced anomalies
output_dir = './Figure3/IPSL/'

GSAT_IPSL_Forced_signal.to_netcdf(output_dir + 'GSAT_IPSL_Forced_anomalies_1850_2022.nc')
GSAT_IPSL_Internal_variability.to_netcdf(output_dir + 'GSAT_IPSL_Internal_Variability_anomalies_1850_2022.nc')